## Laboratorio Final - Equipo Rocket

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py
from collections import Counter
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime as dt
import warnings
warnings.filterwarnings("ignore")
from collections import Counter

In [ ]:
#Cargamos los datos, y eliminamos los que no tienen valor
df = pd.read_csv("calendario_economico.csv") 
df.dropna(inplace = True)

In [ ]:
data2=df.loc[df['Name'] == 'Retail Sales (MoM)']
RetailSales=data2.loc[df['Country'] == 'European Monetary Union']
RetailSales=RetailSales.reset_index(drop=True)

In [ ]:
RetailSales['timestamp']=pd.to_datetime(RetailSales['timestamp'])
RetailSales['timestamp']=RetailSales['timestamp'].dt.tz_localize('UTC')

In [ ]:
#definimos escenarios por restricciones
def escenario(x):
    x['escenario']= x['actual']
    for i in range(len(x)):
        if x['actual'][i] >= x['consensus'][i] >= x['previous'][i]:
            x['escenario'][i] = "A"
        elif x['actual'][i] >= x['consensus'][i] < x['previous'][i]:
            x['escenario'][i]="B"
        elif x['actual'][i] < x['consensus'][i] >= x['previous'][i]:
            x['escenario'][i]="C"
        elif x['actual'][i] < x['consensus'][i] < x['previous'][i]:
            x['escenario'][i]="D"
    return x
    

In [ ]:
# utilizamos la función escenario para obtener cada comunicado del indicador económico si es A, B, C ó D dependiendo del actual,
# previous y consensus.
RS=escenario(RetailSales)

In [ ]:
# Cargamos los datos de los precios del Euro-Dolar
df = pd.read_csv("precios_historicos_eurusd.csv") 
df['timestamp']=pd.to_datetime(df['timestamp'])
df['timestamp']=df['timestamp'].dt.tz_localize('UTC')

In [ ]:
# Checando valores en precios EUR/USD con misma hora que nuestro calendario económico
# Data frame con los valores que coinciden las horas
comparacionRS=df[df.timestamp.isin(RS.timestamp)]

In [ ]:
# Utilizamos compresión de listas para sacar un DataFrame de los indicadores para hacer match entre los comunicados del
# indicador y los precios
#Se busca desde la fecha y hora del indicador agarrar los precios de 30 minutos antes y 30 minutos despues (mas menos 6 periodos)
periodosRS = [df.iloc[i-6:i+7]for i in comparacionRS.index]

In [ ]:
# Con esta función se saca dirección, pips alcistas, pips bajistas y volatilidad de los precios de cada comunicado del indicador
# Se le mete el DataFrame de los comunicados de nuestro indicador y el match de los precios con el indicador
def finalDF(DataFrame,periodos):

    DataFrame['direccion']= DataFrame['actual']
    DataFrame['pips alcistas']=DataFrame['actual']
    DataFrame['pips bajistas']= DataFrame['actual']
    DataFrame['volatilidad']=DataFrame['actual']

    for i in range(len(DataFrame)):

        DataFrame['direccion'][i] = periodos[i]["close"].iloc[-1] - periodos[i]["open"].iloc[6]
        DataFrame['pips alcistas'][i] = periodos[i]["high"].iloc[6:-1].max() - periodos[i]["open"].iloc[0]
        DataFrame['pips bajistas'][i] = periodos[i]["open"].iloc[0] - periodos[i]["low"].iloc[6:-1].min()
        DataFrame['volatilidad'][i] = periodos[i]["high"].max()-periodos[i]["low"].min()
    return DataFrame




In [ ]:
# utilizamos la funcion finalDF() para obtener nuestro DataFrame final en retail sales.
FinalDFRS = finalDF(RS,periodosRS)
FinalDFRS

In [ ]:
#Usamos la funcion Counter para contar cuantas veces se repiten las palabras
newcounter = Counter(FinalDFRS['escenario'])
escenariosRS = pd.DataFrame.from_dict(newcounter, orient='index').reset_index()
escenariosRS.columns = ['escenario', 'cantidad']
escenariosRS
###----------------cuantos escenarios hubo en cada uno A B C y D (RS)-------------------------

In [ ]:
# Sacando cuales informes pertenecen dependiendo de escenario A, B, y C para checar comportamiento
def getescenario(DF):
    escA=DF[DF.escenario.isin(["A"])]
    escB=DF[DF.escenario.isin(["B"])]
    escC=DF[DF.escenario.isin(["C"])]
    escD=DF[DF.escenario.isin(["D"])]
    return escA,escB,escC,escD



In [ ]:
#para sacar el escenario "A"=RSescenarios[0]
#para sacar el escenario B =RSescenarios[1]
#para sacar el escenario C =RSescenarios[2]
#para sacar el escenario D =RSescenarios[3]
RSescenarios=getescenario(FinalDFRS)